In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
import time
import joblib
import os

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# 1. Load and Filter Data
df = pd.read_csv('../data/dataset_1/electrons_train.csv')
df = df[df["p_Truth_isElectron"] == 1]  # Filter only electrons

target = "p_Truth_Energy"
X = df.drop(columns=["p_Truth_isElectron", "p_Truth_Energy"])
y = df[target]

# 2. Train/Val Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# 3. Feature Selection: Top 12 via RF importance
rf_reg = RandomForestRegressor(n_jobs=-1, random_state=42)
rf_reg.fit(X_train, y_train)
feat_importances = pd.Series(rf_reg.feature_importances_, index=X_train.columns)
top_12_features = feat_importances.nlargest(12).index.tolist()

print("Top 12 features:", top_12_features)

'''
# 3. Feature Selection: Top 12 via LightGBM importance
tmp_lgb = lgb.LGBMRegressor(n_estimators=200, random_state=42, n_jobs=-1)
tmp_lgb.fit(X_train, y_train)

feat_importances = pd.Series(tmp_lgb.feature_importances_, index=X_train.columns)
top_12_features = feat_importances.nlargest(12).index.tolist()

print("Top 12 features (LGBM):", top_12_features)
'''

X_train_sel = X_train[top_12_features]
X_val_sel = X_val[top_12_features]

# Optional scaling
scaler = StandardScaler()
X_train_sel = scaler.fit_transform(X_train_sel)
X_val_sel = scaler.transform(X_val_sel)

# 4. Hyperparameter Tuning using Optuna (Bayesian Optimization)
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float("learning_rate", 1e-3, 0.2, log=True),
        'num_leaves': trial.suggest_int("num_leaves", 30, 100),
        'max_depth': trial.suggest_int("max_depth", 16, 24),
        'min_data_in_leaf': trial.suggest_int("min_data_in_leaf", 10, 50),
        'feature_fraction': trial.suggest_float("feature_fraction", 0.5, 1.0),
        'bagging_fraction': trial.suggest_float("bagging_fraction", 0.5, 1.0),
        'bagging_freq': trial.suggest_int("bagging_freq", 1, 7),
        'lambda_l1': trial.suggest_float("lambda_l1", 0, 5),
        'lambda_l2': trial.suggest_float("lambda_l2", 0, 5)
    }

    dtrain = lgb.Dataset(X_train_sel, label=y_train)
    dval = lgb.Dataset(X_val_sel, label=y_val, reference=dtrain)

    gbm = lgb.train(params, train_set=dtrain, valid_sets=[dval], callbacks=[
        lgb.early_stopping(stopping_rounds=7),
        lgb.log_evaluation(0)
    ])
    preds = gbm.predict(X_val_sel)

    rel_mae = np.mean(np.abs((preds - y_val) / y_val))
    return rel_mae


print("⏳ Starting Optuna tuning...")
study = optuna.create_study(direction="minimize")
start_time = time.time()
study.optimize(objective, n_trials=50, timeout=600)
optuna_time = time.time() - start_time

print("✅ Best hyperparameters:", study.best_params)
print(f"⏱️ Tuning time: {optuna_time:.2f} seconds")

# 5. Train final model
best_params = study.best_params
best_params.update({
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1
})

train_data = lgb.Dataset(X_train_sel, label=y_train)
val_data = lgb.Dataset(X_val_sel, label=y_val, reference=train_data)

print("⏳ Training final model...")
start_train_time = time.time()
model = lgb.train(best_params, train_set=train_data, valid_sets=[val_data], callbacks=[
        lgb.early_stopping(stopping_rounds=7),
        lgb.log_evaluation(0)
    ])
train_time = time.time() - start_train_time
print(f"✅ Training time: {train_time:.2f} seconds")



Top 12 features: ['pX_maxEcell_energy', 'pX_ecore', 'p_etcone20', 'pX_E_Lr2_MedG', 'p_ptcone40', 'pX_e233', 'pX_etcone20', 'pX_topoetcone20', 'p_pt_track', 'pX_E3x5_Lr0', 'p_ptcone30', 'p_sigmad0']
⏳ Starting Optuna tuning...
Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[47]	valid_0's rmse: 17716.9
Training until validation scores don't improve for 7 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 17715.8
Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[57]	valid_0's rmse: 18130.6
Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[45]	valid_0's rmse: 17934.7
Training until validation scores don't improve for 7 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 47575.6
Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[54]	valid_0's rmse: 17993.6
Tr

In [35]:
# 6. Evaluate
preds = model.predict(X_val_sel)
rel_mae = np.mean(np.abs((preds - y_val) / y_val))
rmse = mean_squared_error(y_val, preds)
r2 = r2_score(y_val, preds)

print(f"\n📊 Evaluation:")
print(f"Relative MAE: {rel_mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

# 7. Save Model
model_path = "regression_lightgbm_model.json"
model.save_model(model_path)

# Model size
model_size_MB = os.path.getsize(model_path) / (1024**2)
print(f"💾 Saved model size: {model_size_MB:.2f} MB")


def count_split_nodes(node):
    """Recursively counts split nodes in a tree structure."""
    count = 0
    if "split_index" in node:  # This indicates a split node
        count += 1
        # Recursively count in children
        if "left_child" in node:
            count += count_split_nodes(node["left_child"])
        if "right_child" in node:
            count += count_split_nodes(node["right_child"])
    return count

total_split_nodes = 0
for tree_info in model.dump_model()["tree_info"]:
    total_split_nodes += count_split_nodes(tree_info["tree_structure"])

print(f"🧠 Estimated number of split nodes (model params): {total_split_nodes}")
print(f"\n✅ Best hyperparameters: {best_params}")



📊 Evaluation:
Relative MAE: 0.3108
RMSE: 321271897.0914
R²: 0.8970
💾 Saved model size: 0.33 MB
🧠 Estimated number of split nodes (model params): 3841

✅ Best hyperparameters: {'learning_rate': 0.13145544908167586, 'num_leaves': 80, 'max_depth': 17, 'min_data_in_leaf': 35, 'feature_fraction': 0.6346101446829857, 'bagging_fraction': 0.7672496173120773, 'bagging_freq': 6, 'lambda_l1': 0.7106859381792285, 'lambda_l2': 0.6616121995691258, 'objective': 'regression', 'metric': 'rmse', 'verbosity': -1}


In [38]:
import pandas as pd
import numpy as np

# 1. Load test data
test_df = pd.read_csv('./data/dataset_1/AppML_InitialProject_test_regression.csv')

# 2. Select the top 12 features you used during training
# Assuming top_12_features variable exists from before or save/load them
# Here I just use the variable from your code:

# top_12_features = [...]  # Already defined from training

# 3. Prepare test data with selected features
X_test_sel = test_df[top_12_features]

# 4. Apply the same scaler (StandardScaler fitted on training data)
X_test_sel_scaled = scaler.transform(X_test_sel)  # scaler fitted on training data

# 5. Predict with the trained LightGBM model
predictions = model.predict(X_test_sel_scaled)

# 6. Save predictions CSV with index and prediction column
output_df = pd.DataFrame({
    'index': test_df.index,
    'prediction': predictions
})
output_df.to_csv('Regression_HusainManasawala_LightGBM.csv', index=False, header=False)

# 7. Save top 12 feature list CSV
features_df = pd.DataFrame({'top_12_features': top_12_features})
features_df.to_csv('Regression_HusainManasawala_LightGBM_VariableList.csv', index=False, header=False)

# 11. Save variable list
with open('Regression_HusainManasawala_LightGBM_VariableList.csv', 'w') as f:
    for feature in top_12_features:
        f.write(f"{feature},\n")
        
print("✅ Predictions and variable list saved successfully!")


✅ Predictions and variable list saved successfully!


## Now a Neural Net

In [45]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import optuna
import time
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# --- 1. Load and filter data ---
df = pd.read_csv('./data/dataset_1/AppML_InitialProject_train.csv')
df = df[df["p_Truth_isElectron"] == 1]

target = "p_Truth_Energy"
X = df.drop(columns=["p_Truth_isElectron", "p_Truth_Energy"])
y = df[target]

# --- 2. Train/Val split ---
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 3. Feature selection: Top 12 via RF importance ---
rf_reg = RandomForestRegressor(n_jobs=-1, random_state=42)
rf_reg.fit(X_train, y_train)
feat_importances = pd.Series(rf_reg.feature_importances_, index=X_train.columns)
top_12_features = feat_importances.nlargest(12).index.tolist()

print("Top 12 features:", top_12_features)

X_train_sel = X_train[top_12_features]
X_val_sel = X_val[top_12_features]

Top 12 features: ['pX_maxEcell_energy', 'pX_ecore', 'p_etcone20', 'pX_E_Lr2_MedG', 'p_ptcone40', 'pX_e233', 'pX_etcone20', 'pX_topoetcone20', 'p_pt_track', 'pX_E3x5_Lr0', 'p_ptcone30', 'p_sigmad0']


In [ ]:
# --- 4. Scaling ---
scaler = StandardScaler()
X_train_sel = scaler.fit_transform(X_train_sel)
X_val_sel = scaler.transform(X_val_sel)

# Convert to torch tensors (float32)
X_train_t = torch.tensor(X_train_sel, dtype=torch.float32)
y_train_t = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_val_t = torch.tensor(X_val_sel, dtype=torch.float32)
y_val_t = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1)

# --- 5. Setup device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 6. Define neural network ---
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    def forward(self, x):
        return self.net(x)

# --- 7. Training function ---
def train_model(model, train_loader, val_loader, epochs, lr, device):
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val_loss = float('inf')
    best_model_state = None

    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

        # Validation loss
        model.eval()
        val_losses = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                val_loss = criterion(preds, yb)
                val_losses.append(val_loss.item())
        avg_val_loss = np.mean(val_losses)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()

    model.load_state_dict(best_model_state)
    return model

# --- 8. Evaluation function ---
def evaluate(model, X, y, device):
    model.eval()
    with torch.no_grad():
        preds = model(X.to(device)).cpu().numpy().flatten()
    y_true = y.cpu().numpy().flatten()
    rel_mae = np.mean(np.abs((preds - y_true) / y_true))
    rmse = np.sqrt(mean_squared_error(y_true, preds))
    r2 = r2_score(y_true, preds)
    return rel_mae, rmse, r2, preds

# --- 9. Prepare DataLoaders ---
def get_dataloader(X, y, batch_size=64):
    ds = TensorDataset(X, y)
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

train_loader = get_dataloader(X_train_t, y_train_t)
val_loader = get_dataloader(X_val_t, y_val_t, batch_size=256)

# --- 10. Optuna objective ---
def objective(trial):
    hidden_dim = trial.suggest_int("hidden_dim", 128, 300)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    epochs = 50  # fixed for tuning

    model = FeedForwardNN(input_dim=12, hidden_dim=hidden_dim).to(device)
    trained_model = train_model(model, train_loader, val_loader, epochs, lr, device)

    rel_mae, rmse, r2, _ = evaluate(trained_model, X_val_t, y_val_t, device)
    return rel_mae

# --- 11. Run Optuna tuning ---
print("⏳ Starting Optuna tuning...")
study = optuna.create_study(direction="minimize")
start_time = time.time()
study.optimize(objective, n_trials=10, timeout=900)  # 10 trials max 15 min timeout
optuna_time = time.time() - start_time
print(f"✅ Best hyperparameters: {study.best_params}")
print(f"⏱️ Tuning time: {optuna_time:.2f} seconds")

# --- 12. Train final model ---
best_params = study.best_params
best_hidden_dim = best_params["hidden_dim"]
best_lr = best_params["lr"]
epochs_final = 100  # longer training for final

model = FeedForwardNN(input_dim=12, hidden_dim=best_hidden_dim).to(device)
start_train_time = time.time()
model = train_model(model, train_loader, val_loader, epochs_final, best_lr, device)
train_time = time.time() - start_train_time
print(f"✅ Final training time: {train_time:.2f} seconds")

# --- 13. Evaluate final model ---
rel_mae, rmse, r2, preds = evaluate(model, X_val_t, y_val_t, device)
print("\n📊 Final Evaluation:")
print(f"Relative MAE: {rel_mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

# --- 14. Model params count ---
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"🧠 Number of trainable parameters: {total_params}")

# --- 15. Save model ---
model_path = "pytorch_nn_regressor.pt"
torch.save({
    'model_state_dict': model.state_dict(),
    'scaler': scaler,
    'features': top_12_features,
    'params': best_params
}, model_path)

model_size_MB = os.path.getsize(model_path) / (1024**2)
print(f"💾 Saved model size: {model_size_MB:.2f} MB")

Using device: cuda
⏳ Starting Optuna tuning...
✅ Best hyperparameters: {'hidden_dim': 215, 'lr': 0.004446679044470628}
⏱️ Tuning time: 349.04 seconds
✅ Final training time: 83.48 seconds

📊 Final Evaluation:
Relative MAE: 0.3391
RMSE: 18448.2853
R²: 0.8909
🧠 Number of trainable parameters: 142331
💾 Saved model size: 0.55 MB


In [48]:
import pandas as pd
import torch

# 1. Load test data
test_df = pd.read_csv('./data/dataset_1/AppML_InitialProject_test_regression.csv')

# 2. Select the top 12 features you used during training
X_test_sel = test_df[top_12_features]

# 3. Scale using the same scaler fitted on training data
X_test_sel_scaled = scaler.transform(X_test_sel)

# 4. Convert to torch tensor, move to device (assume device defined)
X_test_tensor = torch.tensor(X_test_sel_scaled, dtype=torch.float32).to(device)

# 5. Predict with PyTorch model (eval mode, no grad)
model.eval()
with torch.no_grad():
    preds_tensor = model(X_test_tensor).squeeze().cpu()
predictions = preds_tensor.numpy()

# 6. Save predictions CSV with index and prediction, no header
output_df = pd.DataFrame({
    'index': test_df.index,
    'prediction': predictions
})
output_df.to_csv('Regression_HusainManasawala_PyTorchNN.csv', index=False, header=False)

# 7. Save top 12 features in the comma + newline style you want
with open('Regression_HusainManasawala_PyTorchNN_VariableList.csv', 'w') as f:
    for feature in top_12_features:
        f.write(f"{feature},\n")

print("✅ Predictions and variable list saved successfully!")


✅ Predictions and variable list saved successfully!


/home/husainm97/miniconda3/envs/ml_copenhagen_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


## Trying XGBoost with RandomSearchCV

In [60]:
import pandas as pd
import numpy as np
import xgboost as xgb
import time
import joblib
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 1. Load and Filter Data
df = pd.read_csv('./data/dataset_1/AppML_InitialProject_train.csv')
df = df[df["p_Truth_isElectron"] == 1]  # only electrons

target = "p_Truth_Energy"
X = df.drop(columns=["p_Truth_isElectron", "p_Truth_Energy"])
y = df[target]

# 2. Train/Val Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Feature selection: top 12 with RF
rf_reg = RandomForestRegressor(n_jobs=-1, random_state=42)
rf_reg.fit(X_train, y_train)
feat_importances = pd.Series(rf_reg.feature_importances_, index=X_train.columns)
top_12_features = feat_importances.nlargest(12).index.tolist()
print("Top 12 features:", top_12_features)

X_train_sel = X_train[top_12_features]
X_val_sel = X_val[top_12_features]

# 4. Scaling
scaler = StandardScaler()
X_train_sel = scaler.fit_transform(X_train_sel)
X_val_sel = scaler.transform(X_val_sel)

# 5. Set up XGBoost regressor
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', tree_method='hist', eval_metric='rmse', n_jobs=-1, random_state=42)

# 6. Randomized Search CV hyperparameter space
param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [4, 6, 8, 12, 16],
    'learning_rate': np.logspace(-3, -1, 10),
    'subsample': np.linspace(0.5, 1.0, 6),
    'colsample_bytree': np.linspace(0.5, 1.0, 6),
    'reg_alpha': [0, 0.1, 1, 5],
    'reg_lambda': [0, 0.1, 1, 5],
}

# 7. RandomizedSearchCV
search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_dist,
    n_iter=30,
    scoring='neg_mean_absolute_error',  # using MAE for minimization
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

print("⏳ Starting hyperparameter tuning...")
start_time = time.time()
search.fit(X_train_sel, y_train)
tuning_time = time.time() - start_time
print(f"✅ Tuning done in {tuning_time:.2f} seconds")

best_params = search.best_params_
print("Best params:", best_params)

# 8. Train best model on train+val
X_full = np.vstack([X_train_sel, X_val_sel])
y_full = np.concatenate([y_train, y_val])

best_model = xgb.XGBRegressor(
    **best_params,
    objective='reg:squarederror',
    tree_method='hist',
    eval_metric='rmse',
    n_jobs=-1,
    random_state=42
)

print("⏳ Training final model on full data...")
start_train = time.time()
best_model.fit(X_full, y_full)
train_time = time.time() - start_train
print(f"✅ Training time: {train_time:.2f} seconds")

# 9. Evaluation on validation set (for reference)
val_preds = best_model.predict(X_val_sel)
rel_mae = np.mean(np.abs((val_preds - y_val) / y_val))
rmse = mean_squared_error(y_val, val_preds)
r2 = r2_score(y_val, val_preds)

print("\n📊 Evaluation on validation set:")
print(f"Relative MAE: {rel_mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

# 10. Save model and scaler
model_path = "regression_xgboost_model.json"
best_model.save_model(model_path)
scaler_path = "scaler.joblib"
joblib.dump(scaler, scaler_path)

model_size_MB = os.path.getsize(model_path) / (1024**2)
scaler_size_MB = os.path.getsize(scaler_path) / (1024**2)
print(f"💾 Model size: {model_size_MB:.2f} MB")
print(f"💾 Scaler size: {scaler_size_MB:.2f} MB")


Top 12 features: ['pX_maxEcell_energy', 'pX_ecore', 'p_etcone20', 'pX_E_Lr2_MedG', 'p_ptcone40', 'pX_e233', 'pX_etcone20', 'pX_topoetcone20', 'p_pt_track', 'pX_E3x5_Lr0', 'p_ptcone30', 'p_sigmad0']
⏳ Starting hyperparameter tuning...
Fitting 3 folds for each of 30 candidates, totalling 90 fits
✅ Tuning done in 54.37 seconds
Best params: {'subsample': np.float64(0.9), 'reg_lambda': 5, 'reg_alpha': 1, 'n_estimators': 300, 'max_depth': 8, 'learning_rate': np.float64(0.059948425031894084), 'colsample_bytree': np.float64(0.7)}
⏳ Training final model on full data...
✅ Training time: 0.54 seconds

📊 Evaluation on validation set:
Relative MAE: 0.1969
RMSE: 92228601.9619
R²: 0.9704
💾 Model size: 4.74 MB
💾 Scaler size: 0.00 MB


In [62]:
import pandas as pd

# 1. Load test data
test_df = pd.read_csv('./data/dataset_1/AppML_InitialProject_test_regression.csv')

# 2. Select the top 12 features used during training
X_test_sel = test_df[top_12_features]

# 3. Scale using the same scaler fitted on training data
X_test_sel_scaled = scaler.transform(X_test_sel)

# 4. Predict with XGBoost model
predictions = best_model.predict(X_test_sel_scaled)

# 5. Save predictions CSV with index and prediction, no header
output_df = pd.DataFrame({
    'index': test_df.index,
    'prediction': predictions
})
output_df.to_csv('Regression_HusainManasawala_XGBoost.csv', index=False, header=False)

# 6. Save top 12 features in comma + newline style
with open('Regression_HusainManasawala_XGBoost_VariableList.csv', 'w') as f:
    for feature in top_12_features:
        f.write(f"{feature},\n")

print("✅ Predictions and variable list saved successfully!")


✅ Predictions and variable list saved successfully!


In [64]:
def count_nodes_xgb(tree):
    """Recursively count split nodes and leaf nodes in an XGBoost tree."""
    if 'children' not in tree:
        # Leaf node
        return (0, 1)  # (split_nodes, leaf_nodes)
    else:
        split_nodes = 1  # current split node
        leaf_nodes = 0
        for child in tree['children']:
            s, l = count_nodes_xgb(child)
            split_nodes += s
            leaf_nodes += l
        return (split_nodes, leaf_nodes)

import json

model_dump = xgb_reg.get_booster().get_dump(dump_format='json')

total_splits = 0
total_leaves = 0
for tree_json in model_dump:
    tree_dict = json.loads(tree_json)
    splits, leaves = count_nodes_xgb(tree_dict)
    total_splits += splits
    total_leaves += leaves

total_params = total_splits + total_leaves
print(f"Total split nodes: {total_splits}")
print(f"Total leaf nodes: {total_leaves}")
print(f"Total estimated parameters (splits + leaves): {total_params}")


Total split nodes: 10403
Total leaf nodes: 10603
Total estimated parameters (splits + leaves): 21006


In [54]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import optuna
import time
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# --- 1. Load and filter data ---
df = pd.read_csv('./data/dataset_1/AppML_InitialProject_train.csv')
df = df[df["p_Truth_isElectron"] == 1]

target = "p_Truth_Energy"
X = df.drop(columns=["p_Truth_isElectron", "p_Truth_Energy"])
y = df[target]

# --- 2. Train/Val split ---
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 3. Feature selection: Top 12 via RF importance ---
rf_reg = RandomForestRegressor(n_jobs=-1, random_state=42)
rf_reg.fit(X_train, y_train)
feat_importances = pd.Series(rf_reg.feature_importances_, index=X_train.columns)
top_12_features = feat_importances.nlargest(12).index.tolist()
print("Top 12 features:", top_12_features)

X_train_sel = X_train[top_12_features]
X_val_sel = X_val[top_12_features]

# --- 4. Scaling features ---
scaler = StandardScaler()
X_train_sel = scaler.fit_transform(X_train_sel)
X_val_sel = scaler.transform(X_val_sel)

# --- 4b. Scale target ---
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_val_scaled = target_scaler.transform(y_val.values.reshape(-1, 1))

# --- 5. Convert to torch tensors (float32) ---
X_train_t = torch.tensor(X_train_sel, dtype=torch.float32)
y_train_t = torch.tensor(y_train_scaled, dtype=torch.float32)
X_val_t = torch.tensor(X_val_sel, dtype=torch.float32)
y_val_t = torch.tensor(y_val_scaled, dtype=torch.float32)

# --- 6. Setup device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 7. Define neural network ---
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    def forward(self, x):
        return self.net(x)

# --- 8. Training function ---
def train_model(model, train_loader, val_loader, epochs, lr, device):
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val_loss = float('inf')
    best_model_state = None

    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

        # Validation loss
        model.eval()
        val_losses = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                val_loss = criterion(preds, yb)
                val_losses.append(val_loss.item())
        avg_val_loss = np.mean(val_losses)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()

    model.load_state_dict(best_model_state)
    return model

# --- 9. Evaluation function (with inverse target scaling) ---
def evaluate(model, X, y_scaled, device):
    model.eval()
    with torch.no_grad():
        preds_scaled = model(X.to(device)).cpu().numpy().flatten()
    preds = target_scaler.inverse_transform(preds_scaled.reshape(-1, 1)).flatten()
    y_true = target_scaler.inverse_transform(y_scaled.cpu().numpy().reshape(-1, 1)).flatten()
    rel_mae = np.mean(np.abs((preds - y_true) / y_true))
    rmse = np.sqrt(mean_squared_error(y_true, preds))
    r2 = r2_score(y_true, preds)
    return rel_mae, rmse, r2, preds

# --- 10. Prepare DataLoaders ---
def get_dataloader(X, y, batch_size=64):
    ds = TensorDataset(X, y)
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

train_loader = get_dataloader(X_train_t, y_train_t)
val_loader = get_dataloader(X_val_t, y_val_t, batch_size=256)

# --- 11. Optuna objective ---
def objective(trial):
    hidden_dim = trial.suggest_int("hidden_dim", 128, 300)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    epochs = 50  # fixed for tuning

    model = FeedForwardNN(input_dim=12, hidden_dim=hidden_dim).to(device)
    trained_model = train_model(model, train_loader, val_loader, epochs, lr, device)

    rel_mae, rmse, r2, _ = evaluate(trained_model, X_val_t, y_val_t, device)
    return rel_mae

# --- 12. Run Optuna tuning ---
print("⏳ Starting Optuna tuning...")
study = optuna.create_study(direction="minimize")
start_time = time.time()
study.optimize(objective, n_trials=10, timeout=900)
optuna_time = time.time() - start_time
print(f"✅ Best hyperparameters: {study.best_params}")
print(f"⏱️ Tuning time: {optuna_time:.2f} seconds")

# --- 13. Train final model ---
best_params = study.best_params
best_hidden_dim = best_params["hidden_dim"]
best_lr = best_params["lr"]
epochs_final = 100  # longer training for final

model = FeedForwardNN(input_dim=12, hidden_dim=best_hidden_dim).to(device)
start_train_time = time.time()
model = train_model(model, train_loader, val_loader, epochs_final, best_lr, device)
train_time = time.time() - start_train_time
print(f"✅ Final training time: {train_time:.2f} seconds")

# --- 14. Evaluate final model ---
rel_mae, rmse, r2, preds = evaluate(model, X_val_t, y_val_t, device)
print("\n📊 Final Evaluation:")
print(f"Relative MAE: {rel_mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

# --- 15. Save model ---
model_path = "pytorch_nn_regressor.pt"
torch.save({
    'model_state_dict': model.state_dict(),
    'scaler': scaler,
    'target_scaler': target_scaler,  # Save target scaler for inference!
    'features': top_12_features,
    'params': best_params
}, model_path)

model_size_MB = os.path.getsize(model_path) / (1024**2)
print(f"💾 Saved model size: {model_size_MB:.2f} MB")


Top 12 features: ['pX_maxEcell_energy', 'pX_ecore', 'p_etcone20', 'pX_E_Lr2_MedG', 'p_ptcone40', 'pX_e233', 'pX_etcone20', 'pX_topoetcone20', 'p_pt_track', 'pX_E3x5_Lr0', 'p_ptcone30', 'p_sigmad0']
Using device: cuda
⏳ Starting Optuna tuning...
✅ Best hyperparameters: {'hidden_dim': 224, 'lr': 0.00045677333120969536}
⏱️ Tuning time: 338.20 seconds
✅ Final training time: 69.52 seconds

📊 Final Evaluation:
Relative MAE: 0.3527
RMSE: 18849.1935
R²: 0.8861
💾 Saved model size: 0.40 MB


In [59]:
# --- Load test data ---
df_test = pd.read_csv('./data/dataset_1/AppML_InitialProject_test_regression.csv')
X_test = df_test[top_12_features]

# --- Scale test features ---
X_test_scaled = scaler.transform(X_test)

# --- Convert to torch tensor ---
X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

# --- Predict with model ---
model.eval()
with torch.no_grad():
    preds_scaled = model(X_test_t).cpu().numpy().flatten()

# --- Inverse scale predictions ---
preds = target_scaler.inverse_transform(preds_scaled.reshape(-1, 1)).flatten()

# --- Save predictions ---
output_df = pd.DataFrame({
    "index": df_test.index,          # original index from test data
    "p_Truth_Energy_Pred": preds
})

output_df.to_csv("Regression_HusainManasawala_PyTorchNN.csv", index=False, header=False)
print("✅ Predictions saved to pytorch_nn_predictions.csv")

# --- Save variable list with commas and line breaks ---
with open('Regression_HusainManasawala_PyTorchNN_VariableList.csv', 'w') as f:
    for feature in top_12_features:
        f.write(f"{feature},\n")

print("✅ Variable list saved to Regression_HusainManasawala_PyTorchNN_VariableList.csv")



✅ Predictions saved to pytorch_nn_predictions.csv
✅ Variable list saved to Regression_HusainManasawala_PyTorchNN_VariableList.csv


✅ Predictions saved to pytorch_nn_predictions.csv
